In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
#https://stackoverflow.com/questions/19726663/how-to-save-the-pandas-dataframe-series-data-as-a-figure
import six



In [2]:
tbpath = '../../fits/'
productpath = '../../results/'

In [3]:
rois = []
df = pd.read_csv(tbpath + 'fit_table_reweighted.csv') #get rois in all tables (some may have failed)
rois += list(df.roi.unique())

    
rois = list(set(rois))

In [4]:
#plot side by side 

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
             
roi_us = np.sort([i for i in rois if i[:2]=='US'])#[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])#[::-1]
# roi_other = np.sort(list(pd.read_csv(productpath + "top25.csv").values[:,0])+['China'])
rois = list(roi_us) + list(roi_other)
rois.remove('US')
rois.remove('Gambia')
rois.remove('AA_Global')

In [5]:
theta_ = ["R0","car (week 0)", "ifr (week 0)"]
round_ =[1,2,4,1,2,4]
label_ = {}
label_["R0"] = "R0"
label_["car (week 0)"] = "CAR (week 0)"
label_["ifr (week 0)"] = "IFR (week 0)"
label_["Rlast"] = "R current"
label_["extra_std"] = "Extra Variability"

def afun1(x):
    return '%s' % float('%.1g' %x)

def afun2(x):
    return '%s' % float('%.2g' %x)

df = pd.read_csv(tbpath + 'fit_table_reweighted.csv')
dfreport = pd.DataFrame(columns=
                        ['Region']+
                        ["R0 (CI)"]+
                        ["ci1"]+
                        ["CAR (week 0) (CI)"]+
                        ["ci2"]+
                        ["IFR (week 0) (CI)"]+
                        ["ci3"]+
                        ['Rt(April 15th, 2020) (CI)']+
                        ["ci4"]+
                        ['CARt(April 15th, 2020) (CI)']+
                        ["ci5"]+
                        ['IFRt(April 15th, 2020) (CI)']+
                        ["ci6"]+
                        ["delta weeks"])



k = -1
for roi in rois:
    k += 1
    x = [roi]
    for i in range(len(theta_)):
        theta = theta_[i]
#         print(roi)
#         print(theta)
        mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
        lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
        ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
        if theta == 'R0':
            x += [afun2(mu)]+["("+afun2(lb)+", "+afun2(ub)+")"]
        else:
            x += [afun1(mu)]+["("+afun1(lb)+", "+afun1(ub)+")"]
#         print(x)
    for week in np.arange(11,0,-1):
        theta = 'Rt (week '+str(week)+')'
        x2 = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
        if np.isfinite(x2):
    #             print(x2)
            mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
            x += [afun2(mu)]+["("+afun1(lb)+", "+afun1(ub)+")"]
            break
    for week in np.arange(11,0,-1):
        theta = 'car (week '+str(week)+')'
        x2 = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
        if np.isfinite(x2):
    #             print(x2)
            mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
            x += [afun1(mu)]+["("+afun1(lb)+", "+afun1(ub)+")"]
            break
#     x += [week]
    for week in np.arange(11,0,-1):
        theta = 'ifr (week '+str(week)+')'
        x2 = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
        if np.isfinite(x2):
    #             print(x2)
            mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
            x += [afun1(mu)]+["("+afun1(lb)+", "+afun1(ub)+")"]
            break
    x += [week]
#     print(x)
    try:
        dfreport.loc[k] = x
    except:
        pass
dfreport.to_csv(productpath + "Table1RCarIfr.csv",sep='\t',index=False)
